In [1]:
import pandas as pd
import numpy as np
import sys
import os
from IPython.display import display

Go to https://nsrdb.nrel.gov/data-sets/api-instructions.html for more information on the api and how to get a key.

In [3]:
# Declare all variables as strings. Spaces must be replaced with '+', i.e., change 'John Smith' to 'John+Smith'.
# Define the lat, long of the location and the year
lat, lon, year = 33.2164, -97.1292, 2010
# You must request an NSRDB api key from the link above
api_key = os.getenv('MY_VAR')
# Set the attributes to extract (e.g., dhi, ghi, etc.), separated by commas.
attributes = 'ghi,dhi,dni,wind_speed,air_temperature,solar_zenith_angle'
# Choose year of data
year = '2010'
# Set leap year to true or false. True will return leap day data if present, false will not.
leap_year = 'false'
# Set time interval in minutes, i.e., '30' is half hour intervals. Valid intervals are 30 & 60.
interval = '30'
# Specify Coordinated Universal Time (UTC), 'true' will use UTC, 'false' will use the local time zone of the data.
# NOTE: In order to use the NSRDB data in SAM, you must specify UTC as 'false'. SAM requires the data to be in the
# local time zone.
utc = 'false'
# Your full name, use '+' instead of spaces.
your_name = 'Gage+Krumbach'
# Your reason for using the NSRDB.
reason_for_use = 'beta+testing'
# Your affiliation
your_affiliation = 'my+institution'
# Your email address
your_email = 'gkrumbac@redhat.com'
# Please join our mailing list so we can keep you up-to-date on new developments.
mailing_list = 'false'
# Declare url string
url = 'http://developer.nrel.gov/api/solar/nsrdb_psm3_download.csv?wkt=POINT({lon}%20{lat})&names={year}&leap_day={leap}&interval={interval}&utc={utc}&full_name={name}&email={email}&affiliation={affiliation}&mailing_list={mailing_list}&reason={reason}&api_key={api}'.format(year=year, lat=lat, lon=lon, leap=leap_year, interval=interval, utc=utc, name=your_name, email=your_email, mailing_list=mailing_list, affiliation=your_affiliation, reason=reason_for_use, api=api_key)
# Return just the first 2 lines to get metadata:
info = pd.read_csv(url, nrows=1)
# See metadata for specified properties, e.g., timezone and elevation
timezone, elevation = info['Local Time Zone'], info['Elevation']

i07Zwk93w33ow4twkNxJyA8C5qtgGnc79utnLvjP


In [4]:
# View metadata
pd.set_option('display.max_columns', None)

info

,Source,Location ID,City,State,Country,Latitude,Longitude,Time Zone,Elevation,Local Time Zone,Clearsky DHI Units,Clearsky DNI Units,Clearsky GHI Units,Dew Point Units,DHI Units,DNI Units,GHI Units,Solar Zenith Angle Units,Temperature Units,Pressure Units,Relative Humidity Units,Precipitable Water Units,Wind Direction Units,Wind Speed,Cloud Type -15,Cloud Type 0,Cloud Type 1,Cloud Type 2,Cloud Type 3,Cloud Type 4,Cloud Type 5,Cloud Type 6,Cloud Type 7,Cloud Type 8,Cloud Type 9,Cloud Type 10,Cloud Type 11,Cloud Type 12,Fill Flag 0,Fill Flag 1,Fill Flag 2,Fill Flag 3,Fill Flag 4,Fill Flag 5,Surface Albedo Units,Version
0,NSRDB,693763,-,-,-,33.21,-97.14,-6,203,-6,w/m2,w/m2,w/m2,c,w/m2,w/m2,w/m2,Degree,c,mbar,%,cm,Degrees,m/s,NaN,Clear,Probably Clear,Fog,Water,Super-Cooled Water,Mixed,Opaque Ice,Cirrus,Overlapping,Overshooting,Unknown,Dust,Smoke,NaN,Missing Image,Low Irradiance,Exceeds Clearsky,Missing CLoud Properties,Rayleigh Violation,NaN,v3.0.1


In [5]:
# Return all but first 2 lines of csv to get data:
df = pd.read_csv('http://developer.nrel.gov/api/solar/nsrdb_psm3_download.csv?wkt=POINT({lon}%20{lat})&names={year}&leap_day={leap}&interval={interval}&utc={utc}&full_name={name}&email={email}&affiliation={affiliation}&mailing_list={mailing_list}&reason={reason}&api_key={api}'.format(year=year, lat=lat, lon=lon, leap=leap_year, interval=interval, utc=utc, name=your_name, email=your_email, mailing_list=mailing_list, affiliation=your_affiliation, reason=reason_for_use, api=api_key), skiprows=2)

# Set the time index in the pandas dataframe:
df = df.set_index(pd.date_range('1/1/{yr}'.format(yr=year), freq=interval+'Min', periods=525600/int(interval)))

In [53]:
# take a look

print ('shape:',df.shape)
df.head(120)

shape: (17520, 22)


,Year,Month,Day,Hour,Minute,Temperature,Clearsky DHI,Clearsky DNI,Clearsky GHI,Cloud Type,Dew Point,DHI,DNI,Fill Flag,GHI,Relative Humidity,Solar Zenith Angle,Surface Albedo,Pressure,Precipitable Water,Wind Direction,Wind Speed
2010-01-01 00:00:00,2010,1,1,0,0,-2,0,0,0,0,-3,0,0,0,0,94.14,167.60,0.158,1000.0,0.542,338.4,2.9
2010-01-01 00:30:00,2010,1,1,0,30,-2,0,0,0,0,-3,0,0,0,0,94.16,169.78,0.158,1000.0,0.513,338.4,2.7
2010-01-01 01:00:00,2010,1,1,1,0,-3,0,0,0,0,-4,0,0,0,0,91.27,168.08,0.158,1000.0,0.485,334.6,2.5
2010-01-01 01:30:00,2010,1,1,1,30,-3,0,0,0,0,-4,0,0,0,0,91.29,163.67,0.158,1000.0,0.476,334.6,2.4
2010-01-01 02:00:00,2010,1,1,2,0,-4,0,0,0,0,-4,0,0,0,0,96.25,158.14,0.158,1000.0,0.468,330.0,2.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010-01-03 09:30:00,2010,1,3,9,30,2,99,589,291,4,0,22,0,0,22,91.25,70.98,0.158,1000.0,1.262,136.3,2.3
2010-01-03 10:00:00,2010,1,3,10,0,3,113,649,368,4,0,31,0,0,31,89.36,66.83,0.158,1000.0,1.256,153.3,2.2
2010-01-03 10:30:00,2010,1,3,10,30,4,122,697,436,4,0,132,0,0,132,83.23,63.21,0.158,1000.0,1.246,153.3,1.8
2010-01-03 11:00:00,2010,1,3,11,0,5,130,727,491,4,1,149,0,1,149,81.45,60.23,0.158,1000.0,1.237,181.0,1.5
